In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s6_0gxwy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s6_0gxwy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=047e5d94bdc62461b7cb01567746e6386550b58894ab42e1f278d8655b51c669
  Stored in directory: /tmp/pip-ephem-wheel-cache-iaz_85i7/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<iostream>

using namespace std;

// Kernel definition
__global__ void add(int *a, int *b, int *c) 
{
  *c = *a + *b;
}

int main() 
{

  int a, b, c;
  
  int *d_b, *d_c;
  int size = sizeof(int);
  int *d_a = cudaMalloc(size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);

  c = 0;
  a = 3;
  b = 5;

  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
  
  add<<<1,1>>>(d_a, d_b, d_c);
  cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) 
  {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
  printf("result is %d\n",c);

  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

UsageError: Cell magic `%%cu` not found.


In [9]:
%%cu

#include <stdio.h>

const short N = 10;

__managed__ int a[N], b[N], c[N];

__global__ void Vector_Addition () {
      //Get the id of thread within a block
      
      unsigned short tid = blockDim.x * blockIdx.x + threadIdx.x;

     // check the boundry condition for the threads
      if (tid < N) c[tid] = a[tid] + b[tid];

}


int main (){
      for (int i = 0; i < N; i++){
            a[i] = -i ;
            b[i] = i*i ; 
      }
      int threadsPerBlock = 256;
      int blocksPerGrid =(N + threadsPerBlock - 1) / threadsPerBlock;
      Vector_Addition <<<blocksPerGrid, threadsPerBlock>>> ( ) ;
      cudaDeviceSynchronize();

      for ( int i = 0; i<N; i++ )
      printf("%d + %d = %d\n", a[i] , b[i] , c[i]);

      return 0 ;
}


0 + 0 = 0
-1 + 1 = 0
-2 + 4 = 2
-3 + 9 = 6
-4 + 16 = 12
-5 + 25 = 20
-6 + 36 = 30
-7 + 49 = 42
-8 + 64 = 56
-9 + 81 = 72



In [47]:
%%cu
#include <iostream>
using namespace std;

const short N = 16;

__managed__ int a[N];

__global__ void sum() {
	const int tid = threadIdx.x;

	auto step_size = 1;
	int number_of_threads = blockDim.x;

	while (number_of_threads > 0)
	{
		if (tid < number_of_threads)
		{
			const auto st = tid * step_size * 2;
			const auto en = st + step_size;
      if (st < N && en < N)
			a[st] += a[en];
		}

		step_size <<= 1; 
		number_of_threads >>= 1;
	}
}

int main() {
  int n = N;
  if ((n != 1) && (n & (n - 1))) {
      n |= n >> 1;
      n |= n >> 2;
      n |= n >> 4;
      n |= n >> 8;
      n |= n >> 16;
      n = n + 1;
  }

	for (int i = 0; i < N; i++) a[i] = 1;
	sum <<<1, n / 2 >>>();
  cudaDeviceSynchronize();
  int result = a[0];
	cout << "Sum is " << result << endl;
	return 0;
}

Sum is 16



In [73]:
%%cu
#include <iostream>
#include <algorithm>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sort.h>
#include<thrust/copy.h>
using namespace std;

const int N = 17;

int main() {
  thrust::host_vector<int> h(N);
  for (int i = 0; i < N; i++) h[i] = rand() % 50;
  for (auto to : h) cout << to << ' ';
  cout << '\n';
  thrust::device_vector<int> d = h;
  thrust::sort(d.begin(), d.end());
  thrust::copy(d.begin(), d.end(), h.begin());
  for (auto to : h) cout << to << ' ';
  cout << '\n';
  return 0;
}

33 36 27 15 43 35 36 42 49 21 12 27 40 9 13 26 40 
9 12 13 15 21 26 27 27 33 35 36 36 40 40 42 43 49 



In [72]:
%%cu
#include <iostream>
using namespace std;

const int m = 3, n = 2, p = 4;
__managed__ int a[m * n], b[n * p], c[m * p];

__global__ void MatrixMultiply() {
  int row = blockIdx.x * p + threadIdx.x;
  int col = blockIdx.y * m + threadIdx.y;
  
  if(row < m && col < p) {
    //printf("%d %d\n", row, col);
    float tem = 0;
    for(int k = 0; k < n; k++) {
        tem += a[row * n + k] * b[k * p + col];
    }
    c[row * p + col] = tem;
  }    
}

int main() {
  for (int i = 0; i < m * n; i++) a[i] = rand() % 50;  
  for (int i = 0; i < n * p; i++) b[i] = rand() % 50;
  cout << "Matrix A:\n";
  for (int i = 0; i < m; i++) {
      for (int j = 0; j < n; j++) cout << a[i * n + j] << ' ';
      cout << '\n';
  }  
  cout << "Matrix B:\n";
  for (int i = 0; i < n; i++) {
      for (int j = 0; j < p; j++) cout << b[i * p + j] << ' ';
      cout << '\n';
  } 
  dim3 threadsPerBlock(m, p);
  dim3 blocksPerGrid(1, 1);
  if (m * p > 512){
    threadsPerBlock.x = 512;
    threadsPerBlock.y = 512;
    blocksPerGrid.x = ceil(double(m)/double(threadsPerBlock.x));
    blocksPerGrid.y = ceil(double(p)/double(threadsPerBlock.y));
  }
  MatrixMultiply<<<blocksPerGrid,threadsPerBlock>>>();
  cudaDeviceSynchronize();
  cout << "Matrix C:\n";
  for (int i = 0; i < m; i++) {
      for (int j = 0; j < p; j++) cout << c[i * p + j] << ' ';
      cout << '\n';
  } 
}

Matrix A:
33 36 
27 15 
43 35 
Matrix B:
36 42 49 21 
12 27 40 9 
Matrix C:
1620 2358 3057 1017 
1152 1539 1923 702 
1968 2751 3507 1218 

